<h2>TESTING OPENAI</h2>

In [ ]:

from openai import AzureOpenAI

ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "805387ca-461f-4cb8-af1b-e235da89e5a8"

API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-4o-kenya-hack"

client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION,
)

MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020.",
    },
    {"role": "user", "content": "Where was it played?"},
]

completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=MESSAGES,
)

print(completion.model_dump_json(indent=2))

StatementMeta(, 40cc6b21-6159-4a5c-9335-3b3ac8d8c7af, 6, Finished, Available, Finished)

{
  "id": "chatcmpl-BIGlAULpQkRTnL34ewWx0Iy5FwyQb",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The 2020 World Series was played entirely at **Globe Life Field** in Arlington, Texas, due to the COVID-19 pandemic. It marked the first time in MLB history that the World Series was held at a neutral site. Globe Life Field, the home of the Texas Rangers, provided a controlled environment to minimize travel and exposure risks during the pandemic. The Los Angeles Dodgers defeated the Tampa Bay Rays in six games to claim the title.",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected

<H2></H2>

<H2>STEP 1:WEBSCRAPING, FETCHING AND LOADING  DATA TO POSTGRESQL</H2>

In [22]:
import requests
import pandas as pd
import psycopg2
from bs4 import BeautifulSoup
import re

# PostgreSQL connection parameters
DB_HOST = "hachathonpostgres2025.postgres.database.azure.com"
DB_PORT = "5432"
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "hackathon@2025"

def get_case_links():
    base_urls = [
    
        
      
         "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-devolution-law"
 
    ]
    #     "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-commercial-law",
    #   "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-contract-law",
    #   "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-corporate-law",
    #  "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-criminal-law",
    #    "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-land-law",
    #    "https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-anti-corruption-law"
     #"https://new.kenyalaw.org/taxonomy/case-indexes/case-indexes-common-law"

     
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    all_case_links = []
    
    for base_url in base_urls:
        response = requests.get(base_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve page: {base_url}. Status code: {response.status_code}")
            continue  # Skip to the next URL if there's an error
        
        soup = BeautifulSoup(response.text, "html.parser")
        case_links = ["https://new.kenyalaw.org" + link["href"] for link in soup.find_all("a", href=True) if "/akn/ke/judgment/" in link["href"]]
        
        all_case_links.extend(case_links)
    
    return all_case_links

def scrape_case_data(case_links):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    results = []
    
    for url in case_links:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the case page: {url}. Status code: {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")

        metadata = {}
    
        for dt in soup.find_all("dt"):
            label = dt.text.strip().lower().replace(":", "")  # Normalize label (lowercase, remove colons)
            dd = dt.find_next_sibling("dd")  # Get corresponding <dd> tag
            if dd:
                metadata[label] = dd.text.strip()

        metadata2 = soup.find("dl", class_="document-metadata-list d-md-flex")
        dd_values = [dd.get_text(strip=True) for dd in metadata2.find_all('dd')] if metadata2 else []
        date_published, case_action, court_division, filing_county = (dd_values + [None] * 4)[:4]

        data = {}
        headings = soup.find_all(class_='akn-heading')
        for heading in headings:
            header_text = heading.get_text(strip=True)
            content = []
            next_element = heading.find_next_sibling()
            while next_element and next_element.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                content.append(next_element.get_text(strip=True))
                next_element = next_element.find_next_sibling()
            data[header_text] = ' '.join(content) if content else None

        #ruling_paragraphs = [soup.find(class_='akn-paragraph', id=f'arguments__para_{i}') for i in range(100)]

        ruling_paragraphs = []
        for i in range(50):  # Starting from 1 is more common for paragraph numbering
            # Check for first pattern
            para1 = soup.find(class_='akn-paragraph', id=f'arguments__para_{i}')
            if para1:
                ruling_paragraphs.append(para1)
            
            # Check for second pattern
            para2 = soup.find(class_='akn-paragraph', id=f'arguments__dvs_nn_1__para_{i}')
            if para2:
                ruling_paragraphs.append(para2)

        # Remove any None values that might have been added
        ruling_paragraphs = [p for p in ruling_paragraphs if p is not None]

        #class="akn-div parties-listing"
        #class="akn-div party-name"
        # parties_div = [soup.find('div', class_='akn-div party-name')]
        # parties_title_div = [soup.find_all('div', class_='akn-div')]

        # parties_text = '\n'.join(
        # div.get_text(strip=True, separator='\n')
        # for div in soup.find_all('div', class_='akn-div')
        # if 'parties-listing' in div.get('class', [])
        # ) or ""

        # Combine all content
        # all_content = []
        # all_content = parties_div + parties_title_div + ruling_paragraphs
        # if parties_div:
        #     all_content.append(parties_div.get_text(strip=True, separator='\n'))
    
        # all_content.extend([p.get_text(strip=True, separator='\n') for p in ruling_paragraphs if p]) or None

        # Join all content with double newlines for separation
        #combined_text = '\n\n'.join(all_content)

        ruling_content = " ".join([para.get_text(strip=True) for para in ruling_paragraphs if para]) or None
        
        

        data["Ruling"] = ruling_content

        # print(metadata)

        scraped_data = {
                "Citation": metadata.get("citation"),
                "Court": metadata.get("court"),
                "Court_Station": metadata.get("court station"),
                "Outcome": metadata.get("outcome"),
                "Case_Number": metadata.get("case number"),
                "Judges": metadata.get("judges"),
                "Judgement_Date": metadata.get("judgment date"),
                "Language": metadata.get("language"),
                "Type": metadata.get("type"),
                "Date_Published": date_published,
                "Case_Action": case_action,
                "Court_Division": court_division,
                "Filing_County": filing_county,
                "URL": url
            }
        
        scraped_data.update(data)
        results.append(scraped_data)
    
    return pd.DataFrame(results)

def connect_db():
    """Connects to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        return conn
    except Exception as e:
        print(f"Database connection failed: {e}")
        return None
    
def create_table(sample_data):
    """Creates the kenya_judgments table dynamically based on the columns in sample_data."""
    conn = connect_db()
    if conn is None:
        return
    
    # Extract column names dynamically from sample_data
    column_names = sample_data.columns.tolist()  

    # Define column structure with an "id" as SERIAL PRIMARY KEY and ensure safe column naming
    column_definitions = ["id SERIAL PRIMARY KEY"] + [f'"{col}" TEXT' for col in column_names]

    # SQL queries to drop and create the table
    drop_query = "DROP TABLE IF EXISTS kenya_judgments CASCADE;"
    create_query = f"CREATE TABLE kenya_judgments ({', '.join(column_definitions)});"

    try:
        with conn.cursor() as cur:
            cur.execute(drop_query)  # Drop table if it exists
            cur.execute(create_query)  # Create new table
            conn.commit()
        print("Table dropped (if existed) and created successfully.")
    except Exception as e:
        print(f"Failed to create table: {e}")
    finally:
        conn.close()

def insert_data(dataframe):
    """Inserts data into the kenya_judgments table."""
    conn = connect_db()
    if conn is None:
        return
    
    try:
        with conn.cursor() as cur:
            for _, row in dataframe.iterrows():
                # 🔹 Extract non-empty columns dynamically
                columns = [col for col in dataframe.columns if pd.notna(row[col])]
                values = [row[col] for col in columns]
                
                if not columns:
                    print("Skipping empty row.")
                    continue  # Avoid inserting empty rows

                # 🔹 Safely format column names and placeholders
                columns_str = ', '.join(f'"{col}"' for col in columns)  # Ensure proper SQL syntax
                placeholders = ', '.join(['%s'] * len(values))  # Create (%s, %s, ...) based on column count
                
                query = f'INSERT INTO kenya_judgments ({columns_str}) VALUES ({placeholders}) ON CONFLICT (id) DO NOTHING;'
                
                cur.execute(query, values)
            
            conn.commit()
            print("Data inserted successfully.")
    
    except Exception as e:
        print(f"Failed to insert data: {e}")
    
    finally:
        conn.close()

if __name__ == "__main__":
    case_links = get_case_links()
    if case_links:
        df = scrape_case_data(case_links)
        create_table(df)
        print("Before cleaning:", df.head())  # Debugging step

        # Clean NaN values and standardize column names
        df = df.where(pd.notnull(df), None) 
        # df.rename(columns=COLUMN_MAPPING, inplace=True)
        # df = df[list(COLUMN_MAPPING.values())]  # Keep only mapped columns
        df.to_csv("kenya_law_cases.csv", index=False) 

        print("After cleaning:", df.head())  # Debugging step
        insert_data(df)
        print("Data saved and inserted successfully.")

StatementMeta(, 80114971-79a1-4b8e-a1d1-36d78a06a007, 26, Finished, Available, Finished)

Table dropped (if existed) and created successfully.
Before cleaning:                                             Citation  \
0  Imwatok (MCA) v Nairobi City County & 7 others...   
1  Mutai v Mutai & 2 others; Kirui (Interested Pa...   
2  Council of Governors & another v Attorney Gene...   
3  Opiyo, Chairperson & 5 others v Ayacko Governo...   
4  Karugutiang v County Assembly Service Board We...   

                                   Court  \
0                        Court of Appeal   
1                             High Court   
2                             High Court   
3  Employment and Labour Relations Court   
4  Employment and Labour Relations Court   

                                      Court_Station  \
0                        Court of Appeal at Nairobi   
1                             High Court at Kericho   
2       High Court at Nairobi (Milimani Law Courts)   
3   Employment and Labour Relations Court at Kisumu   
4  Employment and Labour Relations Court at Eldoret  

<h2>STEP 2: ACCESS FABRIC WAREHOUSE DATA AND SUMMARIZE WITH OPEN AI THEN SAVE SUMMARIES</h2>

In [23]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
import pandas as pd
from pyspark.sql import SparkSession

# Azure OpenAI configuration
ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "805387ca-461f-4cb8-af1b-e235da89e5a8"
API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-4o-mini-kenya-hack"

def get_data_from_lakehouse():
    """Retrieve data from Fabric Lakehouse using Delta path"""
    try:
        # Create Spark session
        spark = SparkSession.builder \
            .appName("LakehouseQuery") \
            .getOrCreate()
        
        # Read data from Delta table using the lakehouse path
        #df = spark.read.format("delta").load("/lakehouse/default/tables/SilverTable")
        df = spark.sql("select * from SilverTable where Case_Number not in (select Case_Number from casesummaries)")

        # Convert to pandas DataFrame (if needed)
        pandas_df = df.limit(10).toPandas()  # Get top 10 rows
        
        return pandas_df
    
    except Exception as e:
        print(f"Error connecting to Lakehouse: {str(e)}")
        return None

def generate_summaries(df, text_column):
    """Generate summaries for each row using Azure OpenAI"""
    client = AzureOpenAI(
        azure_endpoint=ENDPOINT,
        api_key=API_KEY,
        api_version=API_VERSION,
    )
    
    summaries = []
    
    for index, row in df.iterrows():
        prompt = f"""
        Analyze the following case data and generate a concise summary:
        
        Case ID: {row['Case_Number']}
        Details: {row[text_column]}
        
        Please provide a 2-3 sentence summary highlighting key points.
        """
        
        try:
            completion = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes case data."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3  # Lower temperature for more factual responses
            )
            
            summary = completion.choices[0].message.content
            summaries.append(summary)
            
        except Exception as e:
            print(f"Error generating summary for Case ID {row['Case_Number']}: {str(e)}")
            summaries.append("Summary unavailable")
    
    return summaries


def save_to_lakehouse_table(df, table_name="CaseSummaries"):
    """Save DataFrame to a new table in the Lakehouse using Spark SQL"""
    try:
        # Create Spark session if not already exists
        spark = SparkSession.builder.getOrCreate()
        
        # Convert pandas DataFrame to Spark DataFrame
        spark_df = spark.createDataFrame(df)
        
        # Save as a new table in the Lakehouse
        spark_df.write.mode("append").format("delta").saveAsTable(table_name)
        
        print(f"Successfully saved data to table: {table_name}")
        
    except Exception as e:
        print(f"Error saving to Lakehouse table: {str(e)}")


def main():
    # Step 1: Get data from Lakehouse
    df = get_data_from_lakehouse()
    
    if df is None or df.empty:
        print("No data retrieved from Lakehouse.")
        return
    
    # Step 2: Specify which column contains the text to summarize
    text_column = "Ruling"  # Replace with your actual column name
    
    # Step 3: Generate summaries
    print("Generating summaries...")
    df['Summary'] = generate_summaries(df, text_column)
    
    # Step 4: Output or save results
    print("\nGenerated Summaries:")
    for _, row in df.iterrows():
        print(f"\nCase ID: {row['Case_Number']}")
        print(f"Summary: {row['Summary']}")
        print("-" * 50)
    
    # Optionally save to CSV
    # df.to_csv('case_summaries.csv', index=False)

    save_to_lakehouse_table(df)

if __name__ == "__main__":
    main()

StatementMeta(, 80114971-79a1-4b8e-a1d1-36d78a06a007, 27, Finished, Available, Finished)

Generating summaries...
Error generating summary for Case ID Environment and Land Appeal E003 of 2023: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 56 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}
Error generating summary for Case ID Environment & Land Case E029 of 2022: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 52 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free